In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing
import seaborn as sns # statistical visualization
import matplotlib.pyplot as plt # graph plotting

In [ ]:
df = pd.read_csv("../input/aula-2-ia-dataset/CasasParaAlugar.csv", index_col=0)
df

# Caracterização das variáveis

Dados categóricos são aqueles delimitados por um conjunto conhecido de opções

Para indentificar no nosso dataset essa característica podemos executar o seguinte script

In [ ]:
features = list(df.head(0))
sorted({feat:df[feat].nunique() for feat in features}.items(),key=lambda item: item[1])

Vemos portanto que os dados `animal`, `furniture` e `city` poderão ser considerados como **categóricos** enquanto os demais colunas serão consideradas como **numéricas**

Para facilitar futuras análises, informamos esse conjunto no momento da importação por meio do argumento `dtype`

In [ ]:
nominal_categorical_features = ['city','animal','furniture']
ordinal_categorical_features = ['bathroom','rooms','parking', 'floor']
categorical_features = nominal_categorical_features+ordinal_categorical_features
numeric_features = list(filter(lambda feat: feat not in categorical_features, features))
df = pd.read_csv("../input/aula-2-ia-dataset/CasasParaAlugar.csv", dtype={feat:'category' for feat in categorical_features}, index_col=0)
df.info()

Com os dados importados e classificados, vamos medir a quantidade de dados ausentes

# Medição

In [ ]:
# Funcao para exibir gráfico de valores ausentes
def graph_missing():
    # Funcao do Pandas usada para contar o numero de valores vazios de cada coluna
    data = df.isna().sum(axis=0)
    y = features
    x = data.values

    # Criamos uma figura
    fig, ax = plt.subplots(figsize=(8, 10))

    # Plota as barras
    ax.barh(y=y, width=x)

    # Adiciona informações no gráfico
    ax.set_yticks(y)
    ax.set_yticklabels(features)
    ax.set_title("Quantidade de variáveis ausentes por coluna")
    plt.show()
    
graph_missing()

# Deleção 

Por meio deste gráfico podemos obversar que em média temos cerca de 1000 registros vazios permeando todas as features

Para nossa análise optou-se que não será necessário a informação de **furniture** (mobília) do nosso dataset, essa escolha foi feita levando em consideração que dados do tipo categórico não podem ser imputados.

In [ ]:
features.remove('furniture')
df = df.loc[:, features]
df

# Imputação de dados ausentes

Iremos utilizar uma modelagem de regrassão para predizer dados de `rent amount` baseando-se em `area`, `property tax` e `total`

In [ ]:
# Criamos um dataframe com os dados de rent amount (R$), area, property tax (R$) e total (R$)
df_regress = pd.concat([df['rent amount (R$)'], df['area'], df['property tax (R$)'], df['total (R$)']], axis=1)
df_regress.head()

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Criamos um objeto que fará a Imputação por Regressão
imp_mean = IterativeImputer(random_state=0)
# Treinamos a regressão com os dados disponiveis
imp_mean.fit(df_regress.values)

# Agora, faremos uma regressão nos mesmos dados usados no treinamento, para
# gerar valores numéricos para substituir os valores ausentes de LotFrontage
X = df_regress.values
regr_output = imp_mean.transform(X)
regr_output

In [ ]:
df['rent amount (R$)'] = regr_output[:, 0]
graph_missing()

Por fim, iremos excluir todos os registros com dados ausentes

In [ ]:
df.dropna(inplace=True)
df.isna().sum()

# Outliers

## Detecção visual de outliers

In [ ]:
selected_features = numeric_features

fig, axes = plt.subplots(ncols=7, figsize=(15, 5))

for i,col in enumerate(selected_features):
    axes[i].boxplot(df[col])
    axes[i].set_title(col)

plt.tight_layout()

Vemos que possuimos dois boxplot muito semelhantes, o de `total (R$)` e o de `area`

Vamos então plotar um gráfico de dispersão para tentar analisar a relação entre essas duas features

In [ ]:
def graph_scatter():
    fig, ax = plt.subplots()

    ax.scatter(x=df['total (R$)'], y=df['area'])
    ax.set_ylabel("area")
    ax.set_xlabel("total (R$)")
    plt.show()
graph_scatter()

### Analisando este gráfico, iremos filtrar:
- `area` <= 3000
- `total (R$)` <= 300000

In [ ]:
print("Tamanho do dataset antes dos filtros: {}".format(df.shape))

mask = df['area'] < 3000
df = df[mask]
mask = df['total (R$)'] < 300000
df = df[mask]

print("Tamanho do dataset depois dos filtros: {}".format(df.shape))

graph_scatter()

Podemos refinar ainda mais diminuindo nosso filtro de `area` para 1250 e `total (R$)` para 30000

In [ ]:
print("Tamanho do dataset antes dos filtros: {}".format(df.shape))

mask = df['area'] < 1250
df = df[mask]
mask = df['total (R$)'] < 30000
df = df[mask]

print("Tamanho do dataset depois dos filtros: {}".format(df.shape))

graph_scatter()